In [ ]:
!pip install gensim nltk

In [3]:

with open('Lil Wayne_lyrics.txt', 'r', encoding='utf-8') as file:
    wayne_text = file.read()

# You might want to print a snippet to verify
print(wayne_text[:500])  # Print the first 500 characters of the text


﻿
I'm on that good kush and alcohol
I got some down bitches I can call
I don't know what I would do without y'all
I'ma ball 'til the day I fall

Yeah, long as my bitches love me (yeah, yeah)
I can give a fuck 'bout no hater
Long as my bitches love me
I can give a fuck 'bout no niggas
Long as these bitches love me

Uh, pussy-ass nigga, stop hatin'
Lil Tunechi got that fire
And these hoes love me like Satan, man
Fuck with me and get bodied
And all she eat is dick; she's on a strict diet
That's my 


In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')  # Download necessary models for tokenization

# Tokenize the text into sentences
sentences = sent_tokenize(wayne_text)

# Tokenize each sentence into words
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]


print(tokenized_sentences[0])





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['\ufeff', 'i', "'m", 'on', 'that', 'good', 'kush', 'and', 'alcohol', 'i', 'got', 'some', 'down', 'bitches', 'i', 'can', 'call', 'i', 'do', "n't", 'know', 'what', 'i', 'would', 'do', 'without', "y'all", "i'ma", 'ball', "'til", 'the', 'day', 'i', 'fall', 'yeah', ',', 'long', 'as', 'my', 'bitches', 'love', 'me', '(', 'yeah', ',', 'yeah', ')', 'i', 'can', 'give', 'a', 'fuck', "'bout", 'no', 'hater', 'long', 'as', 'my', 'bitches', 'love', 'me', 'i', 'can', 'give', 'a', 'fuck', "'bout", 'no', 'niggas', 'long', 'as', 'these', 'bitches', 'love', 'me', 'uh', ',', 'pussy-ass', 'nigga', ',', 'stop', "hatin'", 'lil', 'tunechi', 'got', 'that', 'fire', 'and', 'these', 'hoes', 'love', 'me', 'like', 'satan', ',', 'man', 'fuck', 'with', 'me', 'and', 'get', 'bodied', 'and', 'all', 'she', 'eat', 'is', 'dick', ';', 'she', "'s", 'on', 'a', 'strict', 'diet', 'that', "'s", 'my', 'baby', ',', 'with', 'no', 'makeup', 'she', 'a', 'ten', 'and', 'she', 'the', 'best', 'with', 'that', 'head', 'even', 'better', 'th

In [8]:
from google.colab import files
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Function to read and tokenize a text file
def tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        tokens = word_tokenize(text)
    return tokens

# Upload files
uploaded = files.upload()

# Initialize a list to hold all tokenized texts
all_tokens = []

# Iterate through the uploaded files
for filename in uploaded.keys():
    tokens = tokenize_file(filename)
    all_tokens.append(tokens)

# Train a Word2Vec model
model = Word2Vec(sentences=all_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Save the model
model.save("rap_lyrics_word2vec.model")

# Save the word vectors in a format similar to GloVe
output_file = "glove_format_vectors.txt"
with open(output_file, 'w', encoding='utf-8') as f:
    # Write the header
    vocab_size = len(model.wv)
    vector_size = model.vector_size
    f.write(f"{vocab_size} {vector_size}\n")

    # Write each word and its vector
    for word in model.wv.index_to_key:
        vector = model.wv[word]
        vector_str = ' '.join(map(str, vector))
        f.write(f"{word} {vector_str}\n")

# Download the file
files.download(output_file)


KeyboardInterrupt: 

In [ ]:
!pip install gensim pyphen



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00


In [6]:
import numpy as np
from gensim.models import KeyedVectors
import requests
import random
import string

# Path to the text file containing word vectors
file_path = 'vectorized_weezy_lyrics.txt'

# Function to remove punctuation from words
def clean_word(word):
    return word.translate(str.maketrans('', '', string.punctuation))

# Read the file and load the vectors, cleaning the words
def load_vectors(file_path):
    with open(file_path, 'r') as f:
        # The first line contains the number of vectors and their dimensionality
        num_vectors, vector_size = map(int, f.readline().split())
        word_vectors = KeyedVectors(vector_size)

        for line in f:
            values = line.split()
            word = clean_word(values[0])
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors.add_vector(word, vector)

    return word_vectors

word_vectors = load_vectors(file_path)

# Verify the vectors are loaded correctly
#rint(f"Loaded {len(word_vectors.key_to_index)} word vectors.")

# Function to fetch rhyming words using Datamuse API
def get_rhyming_words(word, num_words=100):
    url = f"https://api.datamuse.com/words?trg_bga_rel={word}&max={num_words}"
    response = requests.get(url)
    rhymes = response.json()
    rhyming_words = [clean_word(rhyme['word']) for rhyme in rhymes if clean_word(rhyme['word']) in word_vectors.key_to_index]
    return rhyming_words

def generate_verse(start_word, num_lines=30, words_per_line=7):
    verse = []
    current_word = clean_word(start_word)
    for _ in range(num_lines):
        line = []
        used_words = set()  # To keep track of words used in the current line
        for _ in range(words_per_line - 1):
            similar_words = word_vectors.most_similar(current_word, topn=50)
            # Filter out words already used in the current line
            filtered_words = [word for word, _ in similar_words if word not in used_words and word != current_word]
            if not filtered_words:
                break  # No more words to select, break out of the loop
            next_word = random.choice(filtered_words)
            line.append(next_word)
            used_words.add(next_word)  # Add the selected word to the used set
            current_word = next_word
        rhyming_words = get_rhyming_words(current_word)
        if rhyming_words:
            line.append(random.choice(rhyming_words))
        else:
            line.append(current_word)
        verse.append(" ".join(line))
    return "\n".join(verse)

start_word = "money"
verse = generate_verse(start_word)
print(verse)

# Save the verse to a file to be used by ChucK
with open("generated_verse.txt", "w") as f:
    f.write(verse)


FileNotFoundError: [Errno 2] No such file or directory: 'vectorized_weezy_lyrics.txt'

In [ ]:
import numpy as np
from gensim.models import KeyedVectors
import requests
import random
import string
import nltk
from nltk.corpus import wordnet as wn

# Download NLTK data files (run this only once)
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

file_path = 'vectorized_weezy_lyrics.txt'

# Function to remove punctuation from words
def clean_word(word):
    return word.translate(str.maketrans(',', '"', string.punctuation))

# Read the file and load the vectors, cleaning the words
def load_vectors(file_path):
    with open(file_path, 'r') as f:
        # The first line contains the number of vectors and their dimensionality
        num_vectors, vector_size = map(int, f.readline().split())
        word_vectors = KeyedVectors(vector_size)

        for line in f:
            values = line.split()
            word = clean_word(values[0])
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors.add_vector(word, vector)

    return word_vectors

word_vectors = load_vectors(file_path)

# Verify the vectors are loaded correctly
#print(f"Loaded {len(word_vectors.key_to_index)} word vectors.")

# Function to fetch rhyming words using Datamuse API
def get_rhyming_words(word, num_words=5000):
    url = f"https://api.datamuse.com/words?rel_rhy={word}&max={num_words}"
    response = requests.get(url)
    rhymes = response.json()
    rhyming_words = [clean_word(rhyme['word']) for rhyme in rhymes if clean_word(rhyme['word']) in word_vectors.key_to_index]
    return rhyming_words

# Function to generate a grammatically coherent line
def generate_line(subject, verb, obj):
    return f"{subject} {verb} {obj}"

# Function to get a word with a specific POS tag
def get_word_with_pos(word, pos_tag, used_words):
    similar_words = word_vectors.most_similar(word, topn=7000)
    for similar_word, _ in similar_words:
        cleaned_word = clean_word(similar_word)
        if nltk.pos_tag([cleaned_word])[0][1] == pos_tag and cleaned_word not in used_words:
            return cleaned_word
    # Fallback to any new similar word if specific POS is not found
    for similar_word, _ in similar_words:
        cleaned_word = clean_word(similar_word)
        if cleaned_word not in used_words:
            return cleaned_word
    return word  # Return the original word if no suitable word is found

def generate_verse(start_word, num_lines=30):
    verse = []
    current_word = clean_word(start_word)
    used_words = set()  # To keep track of all used words

    for _ in range(num_lines):
        # Generate subject, verb, and object
        subject = get_word_with_pos(current_word, 'NN', used_words)
        verb = get_word_with_pos(subject, 'VB', used_words)
        obj = get_word_with_pos(verb, 'NN', used_words)


        # Ensure all words are cleaned
        subject, verb, obj = clean_word(subject), clean_word(verb), clean_word(obj)

        # Generate a grammatically coherent line
        line = generate_line(subject, verb, obj)

        # Find a rhyming word for the object
        rhyming_words = get_rhyming_words(obj)
        if rhyming_words:
            rhyming_word = random.choice(rhyming_words)
            line += f" with {rhyming_word}"
            used_words.add(rhyming_word)  # Add the rhyming word to the used set
        else:
            rhyming_word = None

        verse.append(line)
        current_word = obj  # Update the current word for the next iteration
        used_words.update([subject, verb, obj])  # Add words to the used set

    return "\n".join(verse)

start_word = "love"
verse = generate_verse(start_word)
print(verse)

# Save the verse to a file to be used by ChucK
with open("generated_verse.txt", "w") as f:
    f.write(verse)


 be 
’ see ’
t go nt
niggas know niggas
m do m with em
s get bitch with switch
Cause say love with shove
tell make shit with wit
put have ve with goatee
nigga Tell right with tonight
fuck give fuck with stuck
world let wan with gone
time come pussy
need find ll
Money Do Money with honey
Young See ass with bass
look keep money with funny
re take die with buy
ai Go boy with enjoy
Yeah Call yall
girl leave motherfucker with fucker
want Take turn with burn
Carter run whoa with wo
Mr Look Mr with sister
Hey Believe man with wingspan
today believe ball with wall
lot Give Nigga
head Come head with bread
call Make house with mouse
Weezy Get Weezy
